# Deliverable 3. Create a Travel Itinerary Map

----

1. Create a folder called `Vacation_Itinerary` to store all the files for this deliverable.

2. Download the `Vacation_Itinerary_starter_code.ipynb` file into your `Vacation_Itinerary` folder and rename it `Vacation_Itinerary.ipynb`.

3. Make sure the initial dependencies and the Geoapify API key are imported.

4. From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`.

5. Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

    * The point's size should be the maximum temperature for the city

    * The point's color should be the city's name

    * Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

6. From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

    > **Hint:** You will start and end the route in the same city, so the `vacation_start` and `vacation_end` DataFrames will be in the same city.

7. Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details.

8. Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`.

    > **Hint:** You'll use this DataFrame to create a map using GeoViews, so recall that the first column should be the longitude, and the second the latitude.

9. Use GeoViews to create a map that shows the four cities in the itinerary.

10. Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop.

    > **Hint:** You can note that the `mode` parameter is set to `drive`, you can play around with other modes as it's shown in [the "Travel modes" table](https://apidocs.geoapify.com/docs/routing/#api) in the Geoapify Routing API documentation.

11. Use the Geoapify Routing API to retrieve the route's directions for your itinerary.

12. From the JSON response, store the route's legs coordinates in a variable called `legs`.

13. Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude values into two Python lists named `longitude` and `latitude`.

14. Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`.

15. Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map.

16. Use the asterisk operator to display a composed plot that shows the itinerary's route over the map containing the cities. 

17. Save your map to the `Vacation_Itinerary` folder as `WeatherPy_travel_map.png`.

---

## Make sure the initial dependencies and the Geoapify API key are imported

In [98]:
# Dependencies and Setup
import geoviews as gv
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

## From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`

In [99]:
# Read the WeatherPy_vacation.csv into a DataFrame
vacation_df = vacation_df = pd.read_csv('../Vacation_Search/WeatherPy_vacation.csv')

# Display sample data
vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,3,Avarua,CK,80.65,broken clouds,-21.2078,-159.7750,Paradise Inn
1,5,Isangel,VU,86.11,overcast clouds,-19.5500,169.2667,Tanna Lodge
2,6,Rikitea,PF,77.95,clear sky,-23.1203,-134.9692,Chez Bianca & Benoit
3,11,Gizo,SB,84.92,broken clouds,-8.1030,156.8419,Gizo Hotel
4,18,Kapaa,US,80.92,clear sky,22.0752,-159.3190,Pono Kai Resort


## Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

* The point's size should be the maximum temperature for the city

* The point's color should be the city's name

* Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

In [100]:
# Configure the map plot
map_plot = vacation_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country","Current Description"]
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country,Current Description)

## From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

In [109]:
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method
vacation_start = vacation_df.loc[vacation_df["City"] == "San Juan"]
vacation_end = vacation_start
vacation_stop1 = vacation_df.loc[vacation_df["City"] == "Moron"]
vacation_stop2 = vacation_df.loc[vacation_df["City"] == "Brusque"]
vacation_stop3 = vacation_df.loc[vacation_df["City"] == "Vila Velha"]

## Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details

In [110]:
# Use the Pandas concat function to create a new DataFrame to store the itinerary details.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3, vacation_end], ignore_index=True)

# Display sample data
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,436,San Juan,AR,79.54,broken clouds,-31.5375,-68.5364,Hotel San Francisco
1,454,Moron,AR,83.97,broken clouds,-34.6534,-58.6198,La Esquina
2,430,Brusque,BR,78.10,heavy intensity rain,-27.0981,-48.9175,Hotel Gracher
3,114,Vila Velha,BR,76.95,few clouds,-20.3297,-40.2925,Hotel Prainha
4,436,San Juan,AR,79.54,broken clouds,-31.5375,-68.5364,Hotel San Francisco


## Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`

In [111]:
# Create a Pandas DataFrame to store the latitude and longitude for each city in the itineray
waypoints_df = itinerary_df[["Lat", "Lng"]] 

# Display sample data
waypoints_df

,Lat,Lng
0,-31.5375,-68.5364
1,-34.6534,-58.6198
2,-27.0981,-48.9175
3,-20.3297,-40.2925
4,-31.5375,-68.5364


## Use GeoViews to create map that shows the four cities in the itinerary

In [125]:
# Configure the map plot by using the itineraty_df
waypoints_map = itinerary_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 500,
    frame_height =700,
    size = "Max Temp",
    scale = 2,
    color = "City",
    hover_cols = ["Hotel Name", "Country","Description"]
)

In [126]:
# Display the route_map
waypoints_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country)

## Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop

In [127]:
# Set parameters to trace the route
radius = 5000
params = {
    "mode":"drive",
    "apiKey": geoapify_key,
}

In [128]:
# Set an empty waypoints String variable
waypoints = ""

# Iterate through the route_df DataFrame to define the waypoints
for index, row in waypoints_df.iterrows():
    waypoints = waypoints + str(row["Lat"]) + "," + str(row["Lng"]) + "|"

# Delete the last character from the string
waypoints = waypoints[:-1]

# Add the waypoints to the params dictionary
params["waypoints"] = waypoints

# Display the params dictionary
params

{'mode': 'drive',
 'apiKey': 'f64b9c620b5b4058865c9ae88e5c32c0',
 'waypoints': '-31.5375,-68.5364|-34.6534,-58.6198|-27.0981,-48.9175|-20.3297,-40.2925|-31.5375,-68.5364'}

## Use the Geoapify Routing API to retrieve the route's directions for your itinerary

In [129]:
# Set up the base URL for the Geoapify Places API.
base_url = "https://api.geoapify.com/v1/routing"

# Make request and retrieve the JSON data by using the params dictionaty
route_response = requests.get(base_url, params=params)

# Convert the API response to JSON format
route_response = route_response.json()
display(route_response)

{'features': [{'type': 'Feature',
   'properties': {'mode': 'drive',
    'waypoints': [{'location': [-68.5364, -31.5375], 'original_index': 0},
     {'location': [-58.6198, -34.6534], 'original_index': 1},
     {'location': [-48.9175, -27.0981], 'original_index': 2},
     {'location': [-40.2925, -20.3297], 'original_index': 3},
     {'location': [-68.5364, -31.5375], 'original_index': 4}],
    'units': 'metric',
    'distance': 8186480,
    'distance_units': 'meters',
    'time': 330129.519,
    'legs': [{'distance': 1090725,
      'time': 41582.074,
      'steps': [{'from_index': 0,
        'to_index': 3,
        'distance': 58,
        'time': 5.239,
        'instruction': {'text': 'Drive south on Avenida España.'}},
       {'from_index': 3,
        'to_index': 8,
        'distance': 138,
        'time': 13.828,
        'instruction': {'text': 'Turn left onto Avenida José Ignacio de la Roza.'}},
       {'from_index': 8,
        'to_index': 22,
        'distance': 324,
        'time':

## From the JSON response, store the route's legs coordinates in a variable called `legs`

In [130]:
# Fetch the route's legs coordinates from the JSON reponse
legs = route_response["features"][0]["geometry"]["coordinates"]
display(legs)

[[[-68.536405, -31.5375],
  [-68.536365, -31.537867],
  [-68.536363, -31.537882],
  [-68.536348, -31.538019],
  [-68.536229, -31.538011],
  [-68.536187, -31.538008],
  [-68.535028, -31.53793],
  [-68.535025, -31.53793],
  [-68.534896, -31.537921],
  [-68.534911, -31.537774],
  [-68.534952, -31.537378],
  [-68.534961, -31.537287],
  [-68.534974, -31.537189],
  [-68.535021, -31.536829],
  [-68.535071, -31.536305],
  [-68.535076, -31.536247],
  [-68.535087, -31.53613],
  [-68.535097, -31.536038],
  [-68.535177, -31.535307],
  [-68.535193, -31.535158],
  [-68.535196, -31.535123],
  [-68.535198, -31.535111],
  [-68.535208, -31.535016],
  [-68.535081, -31.535007],
  [-68.53346, -31.534883],
  [-68.533428, -31.534881],
  [-68.533322, -31.534873],
  [-68.533187, -31.534862],
  [-68.532037, -31.534768],
  [-68.532021, -31.534767],
  [-68.531931, -31.53476],
  [-68.53181, -31.534744],
  [-68.531686, -31.534729],
  [-68.531656, -31.534727],
  [-68.531539, -31.534718],
  [-68.531217, -31.534692],


## Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude value into two Python lists names `longitude` and `latitude`

In [131]:
# Create and empty list to store the longitude of each step
longitudes = []

# Create and empty list to store the latitude of step
latitudes = []

# Loop through the legs coordinates to fetch the latitude and longitude for each step
for leg in legs:
    for cordinate in leg:
        latitudes.append(cordinate[1])
        longitudes.append(cordinate[0])

## Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`

In [132]:
# Create an empty DataFrame to store the steps' coordinates
route_df = pd.DataFrame(list(zip(longitudes, latitudes)), columns =['Lng', 'Lat'])

# Add the steps' longitude and latitude from each step as columns to the DataFrame
route_df["Lng"] = longitudes
route_df["Lat"] = latitudes

# Display sample data
route_df.head()

,Lng,Lat
0,-68.536405,-31.537500
1,-68.536365,-31.537867
2,-68.536363,-31.537882
3,-68.536348,-31.538019
4,-68.536229,-31.538011


##  Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map

In [133]:
# Configure the route path by using the GeoViews' Path function
route_path = gv.Path(route_df,).opts(height=500,width=700,line_width = 2.5,color = "Red",legend_position = "top_left")

In [134]:
# Display a composed plot by using the route_map and route_path objects
waypoints_map * route_path

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country)
   .Path.I   :Path   [Lng,Lat]